# Using Transformed Variables in Regression Models

The phrase "feature engineering" refers to methods of transforming the feature variables for a model in order to improve model performance. This lesson discusses feature engineering in the context of linear regression, but the techniques it demonstrates are more broadly applicable.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Load mammals data set
mammals_path = Path('..', 'assets', 'data', 'mammals.txt')
cols = ['brain', 'body']
mammals = pd.read_csv(mammals_path, sep='\t', names=cols, header=0)

**Exercise (10 mins.)**

- Drop mammals with body weight above 200 kg.

- Create a linear regression model of brain weight against body weight: $\mbox{brain} = \beta_0 + \beta_1 * \mbox{body}$

- Superimpose a line plot of the fitted values from that model on a scatterplot of the individual data points.

$\blacksquare$

Linear regression creates a model that is linear in terms of features that you pass into it. **But linear regression can capture non-linear relationships with your original features if you give it non-linear transformations of those features.**

For instance, the code below fits this model:

$\mbox{brain} = \beta_0 + \beta_1 * \mbox{body} + beta_2 * \mbox{body}^2$

In [ ]:
# Re-run the regression with an additional squared term


In [ ]:
# Plot the resulting model on top of the corresponding scatterplot


### Polynomial Terms

A polynomial function of x has the form $c_0 + c_1x + c_2x^2 + c_3x^3 + \ldots$.

If you give a linear regression model $x$, $x^2$, and $x^3$ as features, for instance, it will find the $\beta_0$, $\beta_1$, $\beta_2$, and $\beta_3$ that minimizes mean-squared error for using $\beta_0 + \beta_1x + \beta_2x^2 + \beta_3x^3$ to predict $y$.

It can always recover simple linear regression by setting the coefficients on the higher-order terms to 0, so adding these higher-order terms only increases the set of relationships that the model can capture.

**Exercise (1 min., post answers right away.)**

- How does adding higher-order polynomial terms as inputs to a linear regression model affect its bias and variance?

$\blacksquare$

**Every additional polynomial term gives your model an additional chance to change directions.**

In [ ]:
# first-order
x = np.linspace(-1, 1, 100)
plt.plot(x, x);

In [ ]:
# second-order
plt.plot(x, x**2);

In [ ]:
# third-order
x = np.linspace(-.75, 1.5, 100)
plt.plot(x, -.4*x - x**2 + x**3);

In [ ]:
# fourth-order
x = np.linspace(-1, 1, 100)
plt.plot(x, -x**2 + x**4);

Too many polynomial terms leads to overfitting

In [ ]:
# 8th-order model
g = sns.lmplot(x='body', y='brain', data=mammals, ci=None, order=8);

An (n-1)-order polynomial can always fit n data points perfectly. It is definitely overfitting!

In [ ]:
# 50th-order model
fig = sns.lmplot(x='body', y='brain', data=mammals, ci=None, order=50);
ax = fig.axes
ax[0,0].set_ylim(0, 200);

Including multiple transformations of one variable complicates coefficient interpretation.

In [ ]:
# Print intercept and coefficients from second-order model we created earlier


**Exercise (2 mins., post answers right away)**

- Write down the equation of this second-order model (with fitted coefficient values).

- How would you normally interpret the coefficient on `body` in a linear regression model of brain weight against body weight? Why doesn't that interpretation work in this case?

$\blacksquare$

**sklearn has a "transformer" that generates polynomial terms**

In [ ]:
# sklearn transformers have the same interface as "estimators" (models)
# except that you fit them on features and use them to transform features,
# rather than fitting them on features and a target and using them to predict
# target values.


A transformer returns a modified copy of the object it acts on without changing that objects in place.

**Exercise (10 mins., pair programming)**

Use the Boston housing data for the exercises below.

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['MEDV'])
boston = pd.concat([X, y], axis=1)
boston.head()

- Create a linear regression model for MEDV against DIS with no higher-order polynomial terms.

- Create a linear regression model for MEDV against DIS with polynomial terms for DIS up to and including degree seven.

- **Bonus:** Create line plots of your models' fitted values as a function of DIS and overlay them on scatterplots of MEDV against DIS.

*Tip:* You need to plot your model's predictions against DIS only. If the result looks like spaghetti, then you probably need to sort on DIS. Either sort the DataFrame on DIS from the beginning, or use the `.argsort()` method to get the sequence of row numbers that would sort it on DIS, and use that sequence to select rows for both x and y.

- **Bonus:** Create a model with $DIS$ and $DIS^{-1}$ as features and plot its predictions overlayed on a scatterplot of MEDV against DIS.

$\blacksquare$

**Notes.**

- In statistics, it is extremely unusual to use more than a third-order polynomial. It is more common in machine learning, where the emphasis tends to be on predictive accuracy rather than understanding.
- It would be unusual to use a polynomial term without including all lower-order polynomial terms.
- In addition to polynomial terms (with positive integer exponents), it can also be beneficial to include terms with negative exponents (e.g. $x^{-1}=1/x$) and/or fractional exponents (e.g. $x^{1/2}=\sqrt{X}$).

Let's use 5-fold cross-validation to choose the polynomial order between 1 and 10 that gives the best results in terms of MSE on held-out data, taking advantage of the scikit-learn convenience function `cross_val_score`.

**Exercise (4 mins.)** 

- Create a model with $DIS$ and $DIS^{-1}$ as features and score it using 5-fold cross-validation.

$\blacksquare$

### Interaction Terms

Sometimes the significance of one feature depends on the value of another feature.

For instance, perhaps median housing prices increase as you get closer to a major employment center *unless crime is high around that area*.

We can model these kinds of "interaction effects" by including the *products* of the interacting variables as features in our models.

For example:

$$MEDV = \beta_0 + \beta_1 * DIS + \beta_2 * CRIM + \beta_{12} (DIS * CRIM)$$

In [ ]:
# Implement the model above


**Exercise (2 mins., post answers right away.)**

- Write down the fitted model we just created.

$\blacksquare$

In [ ]:
# check descriptive stats


**Recall the usual interpretation of the coefficient on DIS:** how much the model's prediction for MEDV changes with a one-unit increase in DIS, all else being equal (i.e. for a particular value of CRIM).

**With interaction terms, interpreting the coefficients for a feature DIS requires specifying particular values for the interacting variables.**

For instance, if CRIM is fixed at its 25th percentile value of 0.082, we get

$MEDV = 22.62 + 0.48 * DIS + 0.467 * CRIM - 0.527 (DIS * CRIM)$

$MEDV = 22.62 + 0.48 * DIS + 0.467 * 0.082 - 0.527 (DIS * 0.082)$

$MEDV = 22.62 + 0.48 * DIS + 0.038 - 0.043 * DIS$

$MEDV = 22.658 + 0.437 * DIS$

So **at CRIM=.082**, the model's prediction for MEDV increases by .437 when DIS increases by one. It's better on average to be away from employment centers when crime is low.

The story is different when CRIM has its 75th percentile value of 3.64:

$MEDV = 22.62 + 0.48 * DIS + 0.467 * CRIM - 0.527 (DIS * CRIM)$

$MEDV = 22.62 + 0.48 * DIS + 0.467 * 3.64 - 0.527 (DIS * 3.64)$

$MEDV = 22.62 + 0.48 * DIS + 1.70 - 1.92 * DIS$

$MEDV = 24.32 -1.44 * DIS$

**At CRIM=3.64**, the model's prediction for MEDV *decreases* by 1.44 when DIS increases by one. It's better on average to be close to employment centers when crime is high.

**Exercise (8 mins., pair programming)**

- How does adding interaction terms affect a model's bias and variance?

- Using 5-fold cross-validation, calculate the MSE for a model predicting MEDV from DIS and CRIM without an interaction term. *Hint*: use sklearn.model_selection.cross_val_score

- Using 5-fold cross-validation, calculate the MSE for a model predicting MEDV from DIS and CRIM with an interaction term. *Hint*: use sklearn.model_selection.cross_val_score

- **Bonus:** Find the best model you can, as measured by MSE in 5-fold cross-validation.

$\blacksquare$

### Log Transformations

When your data is very skewed, try a log transformation.

In [ ]:
# Plot histograms of mammal brain and body sizes


In [ ]:
# Plot scatterplot of brain size against body size


In [ ]:
# Plot histograms of mammal brain and body sizes after a log transformation


In [ ]:
# Plot scatterplot of brain size against body size after log transformation


Because we applied a log transformation to $y$ as well as $x$, we need to be careful about how we interpret the MSE values.

In [ ]:
# Train and score a linear model in the original space.
# This model isn't overfitting significantly, so let's not
# worry about a train/test split.


In [ ]:
# Train and score a linear model in the log-transformed space.
# This model isn't overfitting significantly, so let's not
# worry about a train/test split.


Not a fair comparison! MSE for the second model is in log-space.

In [ ]:
# Get MSE for the log-log model in the original space


**What's going on?**

In [ ]:
fix, ax = plt.subplots()
ax.scatter(X, y);
ax.plot(X, np.exp(y_pred_log));

In [ ]:
fix, ax = plt.subplots()
ax.scatter(X, y);
ax.plot(X, y_pred);

The model that we fit in log-log space is getting killed by the points in the top-right:

- MSE punishes large errors.
- Errors that are large in the original space don't look so large in log-log space, so the model doesn't focus on them as much as it "should."

**So which model should you use?** Probably the log-log model.

The original model is better if MSE of brain size prediction is **really** what you care about. But is being off by 1 kg when predicting elephant brain size really as bad as being off by 1 kg when predicting rabbit brain size? Maybe what we really care about is more like *percent error* in terms of number of kg -- that's what a model for the log of brain size optimizes for.

In addition, the log-log model conveys more understanding: modeling log of brain size as a linear function of log of body size plus random noise seems to capture what is really going on.

In [ ]:
print('mean percent error of original model:', ((y-y_pred)/y).mean())
print('mean percent error of model fit in log-log space:',
      ((np.exp(y_log)-np.exp(y_pred_log)/np.exp(y))).mean()
     )

**Takeaways:**

- When your data is highly skewed, try a log transformation.
- When you evaluate a model that transforms $y$, make sure that you calculate metrics for different models on the same scale.
- Make sure that the metric you are optimizing reflects what you care about.

**Notes:**

- A log-transformed variable typically replaces the original variable in a regression analysis, unlike a polynomial term.
- You can apply a log transformation to any combination of your features and your target variable.

### Summary

- Linear regression *can* capture non-linear relationships *when you provide the appropriate non-linear transformations*.
- Every polynomial term you add allows your model to change directions once.
- Log transformations are appropriate for variables with highly skewed distributions.